In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime

In [2]:
main_data_df = pd.read_csv('./prepped_data/메인_데이터_20060101_20210409_ver_2.csv')
print(main_data_df.shape)
main_data_df.head()

(2060316, 14)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,해제사유발생일,부동산유형,전용면적_classes,가격_면적,가격_면적_classes
0,서울특별시 강남구 개포동 1264-3,개포로31길 23-7,(1264-3),53.28,11500.0,-1,1992.0,14.0,2006-08-29,NaN,연립다세대,50~55,215.840841,210~220
1,서울특별시 강남구 개포동 171-13,선릉로14길 11-0,(171-13),68.08,56500.0,2,1988.0,18.0,2006-12-20,NaN,연립다세대,65~70,829.905993,820~830
2,서울특별시 강남구 개포동 1239-7,개포로15길 25-0,강남빌라 가동,52.59,40500.0,2,1988.0,18.0,2006-12-09,NaN,연립다세대,50~55,770.108386,770~780
3,서울특별시 강남구 개포동 1239-7,개포로15길 25-0,강남빌라 가동,52.59,42000.0,1,1988.0,18.0,2006-12-19,NaN,연립다세대,50~55,798.630918,790~800
4,서울특별시 강남구 개포동 1239-6,개포로15길 27-0,강남빌라 나동,52.75,22000.0,2,1988.0,18.0,2006-11-02,NaN,연립다세대,50~55,417.061611,410~420


In [3]:
coor_df = pd.read_csv('./prepped_data/addr_coor_data_ver_1.csv')
print(coor_df.shape)
coor_df.head()

(626610, 4)


,지번주소,도로명주소,X좌표,Y좌표
0,서울특별시 종로구 청운동 1-0,서울특별시 종로구 자하문로36길 16-14,953042.185946,1.954799e+06
1,서울특별시 종로구 청운동 3-0,서울특별시 종로구 자하문로36길 16-11,953033.999264,1.954773e+06
2,서울특별시 종로구 청운동 3-150,서울특별시 종로구 자하문로36길 16-5,953028.717755,1.954717e+06
3,서울특별시 종로구 청운동 4-1,서울특별시 종로구 자하문로36길 28-0,952962.680291,1.954697e+06
4,서울특별시 종로구 청운동 3-94,서울특별시 종로구 자하문로36길 28-0,952962.680291,1.954697e+06


In [4]:
main_data_df.isna().sum()

지번주소                 391
도로명                 7499
건물(단지)명                3
전용면적(㎡)                0
거래금액(만원)               0
층                      0
건축년도                1525
계약날짜기준_건물연식         1525
계약날짜                   0
해제사유발생일          2052740
부동산유형                  0
전용면적_classes           0
가격_면적                  0
가격_면적_classes          0
dtype: int64

In [5]:
both_nan_df = main_data_df[(main_data_df['지번주소'].isna())&(main_data_df['도로명'].isna())]
print(both_nan_df.shape)
both_nan_df.head()

(0, 14)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,해제사유발생일,부동산유형,전용면적_classes,가격_면적,가격_면적_classes


In [6]:
for addr in tqdm(main_data_df['지번주소'].unique()):
    if pd.isna(addr) == True:
        continue
    else:
        addr_df = main_data_df[main_data_df['지번주소'] == addr]
        if addr_df['도로명'].isna().sum() > 0:        
            addr_df['도로명'] = addr_df['도로명'].fillna(method='ffill')
            addr_df['도로명'] = addr_df['도로명'].fillna(method='bfill')

            main_data_df.loc[addr_df.index, '도로명'] = addr_df['도로명']

100%|██████████████████████████████████████████████████████████████████████████| 90602/90602 [1:48:08<00:00, 13.96it/s]


In [7]:
main_data_df.isna().sum()

지번주소                 391
도로명                 4096
건물(단지)명                3
전용면적(㎡)                0
거래금액(만원)               0
층                      0
건축년도                1525
계약날짜기준_건물연식         1525
계약날짜                   0
해제사유발생일          2052740
부동산유형                  0
전용면적_classes           0
가격_면적                  0
가격_면적_classes          0
dtype: int64

In [8]:
for addr in tqdm(main_data_df['도로명'].unique()):
    if pd.isna(addr) == True:
        continue
    else:
        addr_df = main_data_df[main_data_df['도로명'] == addr]
        if addr_df['지번주소'].isna().sum() > 0:        
            addr_df['지번주소'] = addr_df['지번주소'].fillna(method='ffill')
            addr_df['지번주소'] = addr_df['지번주소'].fillna(method='bfill')

            main_data_df.loc[addr_df.index, '지번주소'] = addr_df['지번주소']

100%|██████████████████████████████████████████████████████████████████████████| 91174/91174 [1:50:48<00:00, 13.71it/s]


In [9]:
main_data_df.isna().sum()

지번주소                 391
도로명                 4096
건물(단지)명                3
전용면적(㎡)                0
거래금액(만원)               0
층                      0
건축년도                1525
계약날짜기준_건물연식         1525
계약날짜                   0
해제사유발생일          2052740
부동산유형                  0
전용면적_classes           0
가격_면적                  0
가격_면적_classes          0
dtype: int64

In [10]:
main_data_df.head()

,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,해제사유발생일,부동산유형,전용면적_classes,가격_면적,가격_면적_classes
0,서울특별시 강남구 개포동 1264-3,개포로31길 23-7,(1264-3),53.28,11500.0,-1,1992.0,14.0,2006-08-29,NaN,연립다세대,50~55,215.840841,210~220
1,서울특별시 강남구 개포동 171-13,선릉로14길 11-0,(171-13),68.08,56500.0,2,1988.0,18.0,2006-12-20,NaN,연립다세대,65~70,829.905993,820~830
2,서울특별시 강남구 개포동 1239-7,개포로15길 25-0,강남빌라 가동,52.59,40500.0,2,1988.0,18.0,2006-12-09,NaN,연립다세대,50~55,770.108386,770~780
3,서울특별시 강남구 개포동 1239-7,개포로15길 25-0,강남빌라 가동,52.59,42000.0,1,1988.0,18.0,2006-12-19,NaN,연립다세대,50~55,798.630918,790~800
4,서울특별시 강남구 개포동 1239-6,개포로15길 27-0,강남빌라 나동,52.75,22000.0,2,1988.0,18.0,2006-11-02,NaN,연립다세대,50~55,417.061611,410~420


In [11]:
main_data_df = main_data_df.dropna(subset=['지번주소', '도로명'])
print(main_data_df.shape)
main_data_df.head()

(2055829, 14)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,해제사유발생일,부동산유형,전용면적_classes,가격_면적,가격_면적_classes
0,서울특별시 강남구 개포동 1264-3,개포로31길 23-7,(1264-3),53.28,11500.0,-1,1992.0,14.0,2006-08-29,NaN,연립다세대,50~55,215.840841,210~220
1,서울특별시 강남구 개포동 171-13,선릉로14길 11-0,(171-13),68.08,56500.0,2,1988.0,18.0,2006-12-20,NaN,연립다세대,65~70,829.905993,820~830
2,서울특별시 강남구 개포동 1239-7,개포로15길 25-0,강남빌라 가동,52.59,40500.0,2,1988.0,18.0,2006-12-09,NaN,연립다세대,50~55,770.108386,770~780
3,서울특별시 강남구 개포동 1239-7,개포로15길 25-0,강남빌라 가동,52.59,42000.0,1,1988.0,18.0,2006-12-19,NaN,연립다세대,50~55,798.630918,790~800
4,서울특별시 강남구 개포동 1239-6,개포로15길 27-0,강남빌라 나동,52.75,22000.0,2,1988.0,18.0,2006-11-02,NaN,연립다세대,50~55,417.061611,410~420


In [12]:
main_data_df['도로명주소'] = main_data_df['지번주소'].apply(lambda x: x.split(' ')[0]) + ' '\
+ main_data_df['지번주소'].apply(lambda x: x.split(' ')[1]) + ' ' + main_data_df['도로명']

In [13]:
main_data_df.head()

,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,해제사유발생일,부동산유형,전용면적_classes,가격_면적,가격_면적_classes,도로명주소
0,서울특별시 강남구 개포동 1264-3,개포로31길 23-7,(1264-3),53.28,11500.0,-1,1992.0,14.0,2006-08-29,NaN,연립다세대,50~55,215.840841,210~220,서울특별시 강남구 개포로31길 23-7
1,서울특별시 강남구 개포동 171-13,선릉로14길 11-0,(171-13),68.08,56500.0,2,1988.0,18.0,2006-12-20,NaN,연립다세대,65~70,829.905993,820~830,서울특별시 강남구 선릉로14길 11-0
2,서울특별시 강남구 개포동 1239-7,개포로15길 25-0,강남빌라 가동,52.59,40500.0,2,1988.0,18.0,2006-12-09,NaN,연립다세대,50~55,770.108386,770~780,서울특별시 강남구 개포로15길 25-0
3,서울특별시 강남구 개포동 1239-7,개포로15길 25-0,강남빌라 가동,52.59,42000.0,1,1988.0,18.0,2006-12-19,NaN,연립다세대,50~55,798.630918,790~800,서울특별시 강남구 개포로15길 25-0
4,서울특별시 강남구 개포동 1239-6,개포로15길 27-0,강남빌라 나동,52.75,22000.0,2,1988.0,18.0,2006-11-02,NaN,연립다세대,50~55,417.061611,410~420,서울특별시 강남구 개포로15길 27-0


In [15]:
temp_coor_df = coor_df.drop(columns=['지번주소']).drop_duplicates(subset=['도로명주소'])
print(temp_coor_df.shape)
temp_coor_df.head()

(541117, 3)


,도로명주소,X좌표,Y좌표
0,서울특별시 종로구 자하문로36길 16-14,953042.185946,1.954799e+06
1,서울특별시 종로구 자하문로36길 16-11,953033.999264,1.954773e+06
2,서울특별시 종로구 자하문로36길 16-5,953028.717755,1.954717e+06
3,서울특별시 종로구 자하문로36길 28-0,952962.680291,1.954697e+06
9,서울특별시 종로구 자하문로36길 21-0,953003.226021,1.954652e+06


In [16]:
merge_df = main_data_df.merge(temp_coor_df, on=['도로명주소'])
print(merge_df.shape)
merge_df.head()

(1911851, 17)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,해제사유발생일,부동산유형,전용면적_classes,가격_면적,가격_면적_classes,도로명주소,X좌표,Y좌표
0,서울특별시 강남구 개포동 1264-3,개포로31길 23-7,(1264-3),53.28,11500.0,-1,1992.0,14.0,2006-08-29,NaN,연립다세대,50~55,215.840841,210~220,서울특별시 강남구 개포로31길 23-7,960076.154238,1.942459e+06
1,서울특별시 강남구 개포동 171-13,선릉로14길 11-0,(171-13),68.08,56500.0,2,1988.0,18.0,2006-12-20,NaN,연립다세대,65~70,829.905993,820~830,서울특별시 강남구 선릉로14길 11-0,961145.462320,1.942843e+06
2,서울특별시 강남구 개포동 171-13,선릉로14길 11-0,(171-13),44.80,23300.0,-1,1988.0,28.0,2016-01-16,NaN,연립다세대,40~45,520.089286,520~530,서울특별시 강남구 선릉로14길 11-0,961145.462320,1.942843e+06
3,서울특별시 강남구 개포동 171-13,선릉로14길 11-0,(171-13),44.80,24500.0,-1,1988.0,28.0,2016-04-09,NaN,연립다세대,40~45,546.875000,540~550,서울특별시 강남구 선릉로14길 11-0,961145.462320,1.942843e+06
4,서울특별시 강남구 개포동 171-13,선릉로14길 11-0,(171-13),68.08,60000.0,2,1988.0,31.0,2019-10-23,NaN,연립다세대,65~70,881.316099,880~890,서울특별시 강남구 선릉로14길 11-0,961145.462320,1.942843e+06


In [19]:
%%time
merge_df.to_csv('./prepped_data/메인_데이터_20060101_20210409_ver_3.csv', index=False)

Wall time: 11.2 s
